In [20]:
# compine all steps in Piplines and columntransformer

# Import pandas
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
# Import train_test_split
from sklearn.model_selection import train_test_split
# Import LogisticRegression
from sklearn.linear_model import LogisticRegression

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer

import pickle

In [3]:
# Load dataset
df = pd.read_csv('datasets/cc_approvals.data',header=None)


In [11]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,00202,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,00043,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,00280,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,00100,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,b,21.08,10.085,y,p,e,h,1.25,f,f,0,f,g,00260,0,-
686,a,22.67,0.750,u,g,c,v,2.00,f,t,2,t,g,00200,394,-
687,a,25.25,13.500,y,p,ff,ff,2.00,f,t,1,t,g,00200,1,-
688,b,17.92,0.205,u,g,aa,v,0.04,f,f,0,f,g,00280,750,-


In [ ]:
# function transformer
# to replace '?' with np.nan

def replace_to_nan(df):
    df = df.replace(['?'],np.nan)
    return df


rep_to_nan = FunctionTransformer(replace_to_nan)

def get_columns_dtypes(df):
    

In [9]:
cats = df.select_dtypes(include=['object']).columns
cons = df.select_dtypes(exclude=['object']).columns

In [17]:
trans_null = ColumnTransformer(
    [("numeric_null", SimpleImputer(missing_values='?', strategy='mean'),cons),
    ("cats_null", SimpleImputer(missing_values='?', strategy='most_frequent'),cats)])

In [18]:
x = trans_null.fit_transform(df)

ValueError: 'X' and 'missing_values' types are expected to be both numerical. Got X.dtype=float64 and  type(missing_values)=<class 'str'>.